In [42]:
import pandas as pd

# Load Parquet file
df = pd.read_parquet('pokerdata.parquet')

# Display the first few rows
df.columns

Index(['game', 'hand', 'won', 'firstcard', 'secondcard', 'flop1', 'flop2',
       'flop3', 'turn', 'river',
       ...
       'player6_preflop_check', 'player6_flop_fold', 'player6_flop_bet',
       'player6_flop_check', 'player6_turn_fold', 'player6_turn_bet',
       'player6_turn_check', 'player6_river_fold', 'player6_river_bet',
       'player6_river_check'],
      dtype='object', length=109)

In [43]:
played_hands = df[df['player0_preflop_bet'] > 0]

In [44]:
won_count = played_hands[played_hands['won'] == 1].shape[0]
loss_count = played_hands[played_hands['won'] == 0].shape[0]
print(f"Number of wins: {won_count}")
print(f"Number of losses: {loss_count}")
print(f"Total games: {won_count + loss_count}")
print(f"Win rate: {won_count / (won_count + loss_count) * 100:.2f}%")

Number of wins: 6239
Number of losses: 5379
Total games: 11618
Win rate: 53.70%


In [45]:
print(played_hands.head(100).to_string(index=False))

 game  hand  won firstcard secondcard flop1 flop2 flop3 turn river dealer player0_seat player0_chips player1_seat player1_chips player2_seat player2_chips player3_seat player3_chips player4_seat player4_chips player5_seat player5_chips player6_seat player6_chips  player0_preflop_fold  player0_preflop_bet  player0_preflop_check  player0_flop_fold  player0_flop_bet  player0_flop_check  player0_turn_fold  player0_turn_bet  player0_turn_check  player0_river_fold  player0_river_bet  player0_river_check  player1_preflop_fold  player1_preflop_bet  player1_preflop_check  player1_flop_fold  player1_flop_bet  player1_flop_check  player1_turn_fold  player1_turn_bet  player1_turn_check  player1_river_fold  player1_river_bet  player1_river_check  player2_preflop_fold  player2_preflop_bet  player2_preflop_check  player2_flop_fold  player2_flop_bet  player2_flop_check  player2_turn_fold  player2_turn_bet  player2_turn_check  player2_river_fold  player2_river_bet  player2_river_check  player3_preflop_